In [1]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

In [2]:
def get_page(url):
    # Set up the request headers that we're going to use, to simulate
    # a request by the Chrome browser. Simulating a request from a browser
    # is generally good practice when building a scraper
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
    }

    return requests.get(url, headers=headers)

In [3]:
def parse_rows(table_rows):
    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)
            
    return pd.DataFrame(parsed_rows)

In [4]:
def clean_data(df):
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names
    
    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    
    numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)

    for column_index in range(1, len(df.columns)): # Take all columns, except the first (which is the 'Date' column)
        df.iloc[:,column_index] = df.iloc[:,column_index].str.replace(',', '') # Remove the thousands separator
        df.iloc[:,column_index] = df.iloc[:,column_index].astype(np.float64) # Convert the column to float64
        
    return df

In [5]:
def scrape_table(url):
    # Fetch the page that we're going to parse
    page = get_page(url);

    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want
    tree = html.fromstring(page.content)

    # Fetch all div elements which have class 'D(tbr)'
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
    
    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.
    #assert len(table_rows) > 0
    if len(table_rows) > 0 :
        df = parse_rows(table_rows)
        df = clean_data(df)
        print(df)
        return df
    else:
        return "df_ignore"

In [6]:
companies_list=pd.read_csv('Companies_list.csv')
companies_list.head()

,Unnamed: 0,Symbol,Name
0,0,20MICRONS,20_Microns_Limited
1,1,3IINFOTECH,3i_Infotech_Limited
2,2,3MINDIA,3M_India_Limited
3,3,A2ZMES,A2Z_Maintenance_&_Engineering_Services_Limited
4,4,AANJANEYA,Aanjaneya_Lifecare_Limited


In [7]:
def scrape(symbol):
    print('Attempting to scrape data for ' + symbol)
    try:
        df_balance_sheet = scrape_table('https://in.finance.yahoo.com/quote/'+symbol+ '.NS/balance-sheet?p='+symbol+ '.NS')
    #     if df_balance_sheet != df_ignore:
        df_balance_sheet = df_balance_sheet.set_index('Date')

        df_income_statement = scrape_table('https://in.finance.yahoo.com/quote/'+symbol+ '.NS/financials?p='+symbol+ '.NS')
    #     if df_income_statement != df_ignore:
        df_income_statement = df_income_statement.set_index('Date')

        df_cash_flow = scrape_table('https://in.finance.yahoo.com/quote/'+symbol+ '.NS/cash-flow?p='+symbol+ '.NS')
    #     if df_cash_flow != df_ignore:
        df_cash_flow = df_cash_flow.set_index('Date')

    #     if df_balance_sheet != df_ignore:
        df_joined = df_balance_sheet \
            .join(df_income_statement, on='Date', how='outer', rsuffix=' - Income Statement') \
            .join(df_cash_flow, on='Date', how='outer', rsuffix=' - Cash Flow') \
            .dropna(axis=1, how='all') \
            .reset_index()

        df_joined.insert(1, 'Symbol', symbol)

        return df_joined
    except: 
        pass

In [8]:
symbols = companies_list['Symbol']
symbols.values

array(['20MICRONS', '3IINFOTECH', '3MINDIA', ..., 'ZUARIAGRO',
       'ZYDUSWELL', 'ZYLOG'], dtype=object)

In [9]:
def scrape_multi(symbols):
    return pd.concat([scrape(symbol) for symbol in symbols], sort=False)

In [10]:
#!pip install tqdm
#pip list | findstr /I tqdm
from tqdm import tqdm_notebook
tqdm=tqdm_notebook

In [11]:
import time

In [28]:
lis_df = []
for sym in tqdm(symbols):
    lis_df.append(scrape(sym))
    time.sleep(7) 

<ipython-input-28-4bde4b88d45e>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sym in tqdm(symbols):


Attempting to scrape data for 20MICRONS
        Date  Assets  Current assets  Cash  Cash and cash equivalents  \
1  31/3/2020     NaN             NaN   NaN                    61057.0   
2  31/3/2019     NaN             NaN   NaN                    70870.0   
3  31/3/2018     NaN             NaN   NaN                    60544.0   

   Other short-term investments  Total cash  Net receivables  Inventory  \
1                       14677.0     75734.0         820991.0   977218.0   
2                       34585.0    105455.0         696112.0   836289.0   
3                       17178.0     77722.0         664814.0   722321.0   

   Other current assets  ...  Long-term debt  Deferred tax liabilities  \
1               25604.0  ...        389234.0                  311325.0   
2                  21.0  ...        456608.0                  287800.0   
3                  18.0  ...        581445.0                  240294.0   

   Other long-term liabilities  Total non-current liabilities  \
1   

        Date  Total revenue  Cost of revenue  Gross profit  \
1        ttm     10415500.0        7324700.0     3090800.0   
2  31/3/2020     11413200.0        8071200.0     3342000.0   
3  31/3/2019     11217500.0        8218100.0     2999400.0   
4  31/3/2018      9911900.0        7096300.0     2815600.0   

   Selling general and administrative  Total operating expenses  \
1                                 NaN                 1453100.0   
2                            503400.0                 1748300.0   
3                            449400.0                 1587500.0   
4                            335300.0                 1296700.0   

   Operating income or loss  Interest expense  Income before tax  \
1                 1637700.0          833200.0           851000.0   
2                 1593700.0          874900.0           814600.0   
3                 1411900.0          816600.0           791100.0   
4                 1518900.0          840300.0           794800.0   

   Income ta

        Date  Net income  Depreciation & amortisation  \
1        ttm   1520207.0                          NaN   
2  31/3/2020   3221692.0                     593325.0   
3  31/3/2019   3661799.0                     437563.0   
4  31/3/2018   3105356.0                     418032.0   

   Change in working capital  Inventory  Other working capital  \
1                        NaN        NaN                    NaN   
2                  -919985.0  -273929.0              2238169.0   
3                  -712049.0  -437886.0              2110545.0   
4                 -2659342.0  -155113.0               136592.0   

   Other non-cash items  Net cash provided by operating activities  \
1                   NaN                                        NaN   
2             -376753.0                                  2450467.0   
3             -351917.0                                  2529817.0   
4             -392588.0                                   294200.0   

   Investments in property, plan

Attempting to scrape data for AARTIIND
        Date  Assets  Current assets  Cash  Cash and cash equivalents  \
1  31/3/2020     NaN             NaN   NaN                  2449600.0   
2  31/3/2019     NaN             NaN   NaN                  8021500.0   
3  31/3/2018     NaN             NaN   NaN                   301100.0   

   Total cash  Net receivables  Inventory  Other current assets  \
1   2449600.0        7534400.0  8356800.0                   NaN   
2   8021500.0        7760400.0  7717800.0                 100.0   
3    301100.0        5907500.0  7472900.0                 100.0   

   Total current assets  ...  Long-term debt  Deferred tax liabilities  \
1            20048700.0  ...       5808400.0                 2110100.0   
2            25771100.0  ...       8148000.0                 1930100.0   
3            16209400.0  ...       9082700.0                 1774100.0   

   Deferred revenues  Total non-current liabilities  Total liabilities  \
1          5296900.0        

...

        Date  Net income  Depreciation & amortisation  \
1        ttm   1185621.0                          NaN   
2  31/3/2020   1423621.0                     824886.0   
3  31/3/2019   1443997.0                     776887.0   
4  31/3/2018   1231744.0                     688640.0   

   Change in working capital  Inventory  Other working capital  \
1                        NaN        NaN                    NaN   
2                   116663.0   -12623.0              1186691.0   
3                  -719163.0   -20080.0               539314.0   
4                  -342473.0    -7937.0               210103.0   

   Other non-cash items  Net cash provided by operating activities  \
1                   NaN                                        NaN   
2              298350.0                                  2522688.0   
3              330880.0                                  1830405.0   
4              310938.0                                  1801243.0   

   Investments in property, plan

        Date  Assets  Current assets  Cash  Cash and cash equivalents  \
1  31/3/2020     NaN             NaN   NaN                   268200.0   
2  31/3/2019     NaN             NaN   NaN                   219400.0   
3  31/3/2018     NaN             NaN   NaN                   459600.0   

   Total cash  Net receivables  Inventory  Total current assets  \
1    268200.0        1340300.0  3872000.0             6575200.0   
2    219400.0        1583700.0  4025900.0             7616100.0   
3    459600.0        1923900.0  4119100.0             8275800.0   

   Non-current assets  ...  Deferred tax liabilities  Deferred revenues  \
1                 NaN  ...                 4123400.0          6164100.0   
2                 NaN  ...                 4123400.0          5651600.0   
3                 NaN  ...                 4123400.0          4089300.0   

   Other long-term liabilities  Total non-current liabilities  \
1                       -200.0                     88680400.0   
2      

        Date  Net income  Depreciation & amortisation  \
1        ttm    668656.0                          NaN   
2  31/3/2020    516256.0                     292356.0   
3  31/3/2019    581191.0                     266322.0   
4  31/3/2018    169965.0                     241555.0   

   Change in working capital  Inventory  Other working capital  \
1                        NaN        NaN                    NaN   
2                 -1518538.0   584740.0              -365932.0   
3                  -289279.0 -2680045.0               907239.0   
4                  -656249.0 -1094931.0               -15055.0   

   Other non-cash items  Net cash provided by operating activities  \
1                   NaN                                        NaN   
2              713884.0                                   145293.0   
3              732859.0                                  1431109.0   
4              765148.0                                   507196.0   

   Investments in property, plan

Attempting to scrape data for VAIBHAVGEM
Attempting to scrape data for VAKRANSOFT
Attempting to scrape data for VALECHAENG
Attempting to scrape data for VALUEIND
Attempting to scrape data for VARDHACRLC
        Date  Assets  Current assets  Cash  Cash and cash equivalents  \
1  31/3/2020     NaN             NaN   NaN                   627964.0   
2  31/3/2019     NaN             NaN   NaN                    17670.0   
3  31/3/2018     NaN             NaN   NaN                    40930.0   

   Other short-term investments  Total cash  Net receivables  Inventory  \
1                     1578844.0   2206808.0          95041.0   604702.0   
2                     2256624.0   2274294.0         156029.0   763717.0   
3                     1414432.0   1455362.0         202153.0   524652.0   

   Other current assets  ...  Long-term debt  Deferred tax liabilities  \
1                   NaN  ...             0.0                  105612.0   
2               39024.0  ...             0.0           

        Date  Total revenue  Cost of revenue  Gross profit  \
1        ttm      4501404.0        3447236.0     1054168.0   
2  31/3/2020      5608650.0        4373949.0     1234701.0   
3  31/3/2019      8107064.0        6286811.0     1820253.0   
4  31/3/2018      6356693.0        5079696.0     1276997.0   

   Selling general and administrative  Total operating expenses  \
1                                 NaN                 1181483.0   
2                             75299.0                 1386694.0   
3                             86781.0                 1978309.0   
4                            164990.0                 2185086.0   

   Operating income or loss  Interest expense  Income before tax  \
1                 -127315.0          575285.0          1060956.0   
2                 -151993.0          596947.0          1018428.0   
3                 -158056.0          599326.0          -739812.0   
4                 -908089.0          808520.0         -1716552.0   

   Income ta

In [42]:
len(lis_df)
print(lis_df)

RangeIndex(start=0, stop=1, step=1)


In [30]:
lis_df_1 = [i for i in lis_df if i is not None]
len(lis_df_1)


250

In [51]:
lis_df_main=pd.concat(objs=lis_df_1[0:100])
#lis_df_main.shape

In [ ]:
#scrape("3MINDIA")

In [ ]:
#main_df = scrape_multi(symbols)
#main_df.head(10)

In [52]:
date = datetime.today().strftime('%Y-%m-%d')
writer = pd.ExcelWriter('Yahoo-Finance-Scrape-' + date + '.xlsx')
lis_df_main.to_excel(writer)
writer.save()


In [53]:
temp=pd.read_excel("Yahoo-Finance-Scrape-2021-05-14.xlsx")

In [54]:
temp.describe

<bound method NDFrame.describe of      Unnamed: 0      index      Symbol       Date  Cash and cash equivalents  \
0             0  31/3/2020   20MICRONS  31/3/2020                    61057.0   
1             1  31/3/2019   20MICRONS  31/3/2019                    70870.0   
2             2  31/3/2018   20MICRONS  31/3/2018                    60544.0   
3             3        NaN   20MICRONS        ttm                        NaN   
4             0  31/3/2020  3IINFOTECH  31/3/2020                   895300.0   
..          ...        ...         ...        ...                        ...   
403           3        NaN        GLFL        ttm                        NaN   
404           0  31/3/2020   GLOBUSSPR  31/3/2020                   199846.0   
405           1  31/3/2019   GLOBUSSPR  31/3/2019                    16012.0   
406           2  31/3/2018   GLOBUSSPR  31/3/2018                    17179.0   
407           3        NaN   GLOBUSSPR        ttm                        NaN   

     

In [43]:
l=(np.array(list(temp.iloc[0])))
l

array(['0', '31/3/2020', '20MICRONS', '31/3/2020', '61057.0', '14677.0',
       '75734.0', '820991.0', '977218.0', '25604.0', '2163439.0',
       '3169114.0', '-1111513.0', '2057601.0', '216.0', '62613.0', 'nan',
       '2283335.0', '4446774.0', '802454.0', '879710.0', '-1.0',
       '1821295.0', '389234.0', '311325.0', 'nan', '739949.0',
       '2561244.0', '176433.0', '1281665.0', '1878032.0', '4446774.0',
       '5291901', '3415334', '1876567', '764389.0', '1331373', '545194',
       '172380', '345536', '103775', '241761', '241171', '241171',
       '35287.0', '35287.0', '643023.0', '241171', '125107', '-142388',
       '-140929', '262269', '207691', '428215', '-165946', '0.0', '0',
       '13590.0', 'nan', '-140119', '-13970', '70774', '56805', '428215',
       '-165946', '262269'], dtype='<U21')

In [67]:
data={'index':['0'], 'Symbol':['0'], 'Date':['0'], 'Cash and cash equivalents':['0'],
       'Other short-term investments':['0'], 'Total cash':['0'], 'Net receivables':['0'],
       'Inventory':['0'], 'Other current assets':['0'], 'Total current assets':['0'],
       'Gross property, plant and equipment':['0'], 'Accumulated depreciation':['0'],
       'Net property, plant and equipment':['0'], 'Goodwill':['0'], 'Intangible assets':['0'],
       'Other long-term assets':['0'], 'Total non-current assets':['0'], 'Total assets':['0'],
       'Current debt':['0'], 'Accounts payable':['0'], 'Other current liabilities':['0'],
       'Total current liabilities':['0'], 'Long-term debt':['0'],
       'Deferred tax liabilities':['0'], 'Other long-term liabilities':['0'],
       'Total non-current liabilities':['0'], 'Total liabilities':['0'], 'Common stock':['0'],
       'Retained earnings':['0'], "Total stockholders' equity":['0'],
       "Total liabilities and stockholders' equity":['0'], 'Total revenue':['0'],
       'Cost of revenue':['0'], 'Gross profit':['0'], 'Selling general and administrative':['0'],
       'Total operating expenses':['0'], 'Operating income or loss':['0'],
       'Interest expense':['0'], 'Income before tax':['0'], 'Income tax expense':['0'],
       'Income from continuing operations':['0'], 'Net income':['0'],
       'Net income available to common shareholders':['0'], 'Basic average shares':['0'],
       'Diluted average shares':['0'], 'EBITDA':['0'], 'Net income - Cash Flow':['0'],
       'Depreciation & amortisation':['0'], 'Change in working capital':['0'],
       'Inventory - Cash Flow':['0'], 'Other working capital':['0'],
       'Other non-cash items':['0'], 'Net cash provided by operating activities':['0'],
       'Investments in property, plant and equipment':['0'], 'Acquisitions, net':['0'],
       'Purchases of investments':['0'], 'Sales/maturities of investments':['0'],
       'Other investing activities':['0'], 'Net cash used for investing activities':['0'],
       'Net change in cash':['0'], 'Cash at beginning of period':['0'],
       'Cash at end of period':['0'], 'Operating cash flow':['0'], 'Capital expenditure':['0'],
       'Free cash flow':['0']}
df_ignore=pd.DataFrame(data)

In [68]:
df_ignore

,index,Symbol,Date,Cash and cash equivalents,Other short-term investments,Total cash,Net receivables,Inventory,Other current assets,Total current assets,...,Purchases of investments,Sales/maturities of investments,Other investing activities,Net cash used for investing activities,Net change in cash,Cash at beginning of period,Cash at end of period,Operating cash flow,Capital expenditure,Free cash flow
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
